In [ ]:
from fastai2.text.all import *

In [ ]:
from nbdev.showdoc import *

In [ ]:
# all_slow

# Transfer learning in text

> How to fine-tune a language model and train a classifier

## Finetune a pretrained Language Model

First we get our data and tokenize it.

In [ ]:
path = untar_data(URLs.IMDB_SAMPLE)
df = pd.read_csv(path/'texts.csv')

Then we put it in a `DataSource`. For a language model, we don't have targets, so there is only one transform to numericalize the texts.

In [ ]:
splits = ColSplitter()(df)
tfms = [attrgetter("text"), Tokenizer.from_df("text"), Numericalize()]
dsrc = DataSource(df, [tfms], splits=splits, dl_type=LMDataLoader)

Then we use that `DataSource` to create a `DataBunch`. Here the class of `TfmdDL` we need to use is `LMDataLoader` which will concatenate all the texts in a source (with a shuffle at each epoch for the training set), split it in `bs` chunks then read continuously through it.

In [ ]:
dbunch = dsrc.databunch(bs=64, seq_len=72)

Or more simply with a factory method:

In [ ]:
dbunch = TextDataBunch.from_df(df, text_col='text', is_lm=True, valid_col='is_valid')

In [ ]:
dbunch.show_batch(max_n=2)

,text,text_
0,"xxbos xxmaj this is a extremely well - made film . xxmaj the acting , script and camera - work are all first - rate . xxmaj the music is good , too , though it is mostly early in the film , when things are still relatively xxunk . xxmaj there are no really xxunk in the cast , though several faces will be familiar . xxmaj the entire cast does","xxmaj this is a extremely well - made film . xxmaj the acting , script and camera - work are all first - rate . xxmaj the music is good , too , though it is mostly early in the film , when things are still relatively xxunk . xxmaj there are no really xxunk in the cast , though several faces will be familiar . xxmaj the entire cast does an"
1,". xxmaj before i go further , it seems only fair that i point out the following so a reader can see if xxmaj i 'm xxunk or not . xxmaj i 'm trying to be objective , for the record . \n\n i tend to enjoy xxmaj maher 's xxup xxunk show now and then , though i rarely think he 's the source of the humor . i do n't","xxmaj before i go further , it seems only fair that i point out the following so a reader can see if xxmaj i 'm xxunk or not . xxmaj i 'm trying to be objective , for the record . \n\n i tend to enjoy xxmaj maher 's xxup xxunk show now and then , though i rarely think he 's the source of the humor . i do n't really"


Then we have a convenience method to directly grab a `Learner` from it, using the `AWD_LSTM` architecture.

In [ ]:
learn = language_model_learner(dbunch, AWD_LSTM, metrics=[accuracy, Perplexity()], path=path, opt_func = partial(Adam, wd=0.1)).to_fp16()

In [ ]:
learn.freeze()
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.537915,4.057219,0.274963,57.813309,00:09


In [ ]:
learn.unfreeze()
learn.fit_one_cycle(4, 1e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.315029,4.153787,0.261169,63.674690,00:11
1,4.140304,4.086970,0.268618,59.559143,00:11
2,3.815153,4.053044,0.273681,57.572433,00:11
3,3.428173,4.084964,0.272838,59.439785,00:10


Once we have fine-tuned the pretrained language model to this corpus, we save the encoder since we will use it for the classifier.

In [ ]:
learn.show_results()

,input,target,pred
0,"xxbos i found myself very caught up in this movie , at least at the beginning , and any credit i give to this movie , is xxmaj xxunk xxmaj xxunk , she was fantastic ! ! xxmaj but that s where it ends . i seem to be very good at xxunk out who the killer is , and i like it when a movie is able to completely xxunk me","i found myself very caught up in this movie , at least at the beginning , and any credit i give to this movie , is xxmaj xxunk xxmaj xxunk , she was fantastic ! ! xxmaj but that s where it ends . i seem to be very good at xxunk out who the killer is , and i like it when a movie is able to completely xxunk me ,","xxmaj was this watching excited up in this movie . and least at the beginning , but i bit i had to the movie is is that xxunk xxmaj xxunk . the was the . xxmaj xxmaj the i 's not it was ! xxmaj think to be a disappointed at watching out the i movie was , but i think to . i movie is shot to make xxunk the ."
1,"logic , not only is the acting terrible , not only is the entire movie offensive from start to finish , not only is the direction as amateurish as you can find , but they actually want you to pay to see this film . xxmaj maybe if it was free … n xxrep 3 a h , it would still be a waste of time . \n\n xxmaj usually xxmaj i",", not only is the acting terrible , not only is the entire movie offensive from start to finish , not only is the direction as amateurish as you can find , but they actually want you to pay to see this film . xxmaj maybe if it was free … n xxrep 3 a h , it would still be a waste of time . \n\n xxmaj usually xxmaj i 'd",", and only is the film as , but only is the acting movie just , start to finish , but only is the acting and xxunk as it can find , but also also have to to see for see this movie . xxmaj it it you was a , i xxrep 3 o xxunk xxrep it would have be a little of time . xxbos xxmaj the i i 'm"
2,"was xxunk , cliché , or just plain stupid . xxmaj for instance the name of the camp is "" camp blood "" ( lame ) , the name of the clown is "" the killer clown "" ( lame ) . xxmaj what is a clown doing in a forest anyway ? xxmaj was that the only mask they could find ? 3 . xxmaj the last but certainly the least","xxunk , cliché , or just plain stupid . xxmaj for instance the name of the camp is "" camp blood "" ( lame ) , the name of the clown is "" the killer clown "" ( lame ) . xxmaj what is a clown doing in a forest anyway ? xxmaj was that the only mask they could find ? 3 . xxmaj the last but certainly the least was","a by and - and not plain stupid . xxmaj the me , xxunk of the film is "" xxunk "" "" . xxunk ) . but word of the film is "" the xxmaj "" "" ( xxunk ) . xxmaj the makes the joke doing in a car is ? xxmaj and that the name thing you could make ? xxmaj . xxmaj the xxunk scene xxunk the worst was"
3,"wife 's teeth and cut out her tongue with no xxunk ? xxmaj overall awesome flick , but not for everyone . xxbos xxmaj red xxmaj rock xxmaj west is one of those rare films that keeps you guessing the entire time as to what will happen next . xxmaj nicolas xxmaj cage is mistaken for a contract killer as he enters a small town trying to find work . xxmaj dennis","'s teeth and cut out her tongue with no xxunk ? xxmaj overall awesome flick , but not for everyone . xxbos xxmaj red xxmaj rock xxmaj west is one of those rare films that keeps you guessing the entire time as to what will happen next . xxmaj nicolas xxmaj cage is mistaken for a contract killer as he enters a small town trying to find work . xxmaj dennis xxmaj","and daughter , her out her hair . a xxunk . xxmaj the , movie . but it as me . xxmaj xxmaj this xxmaj rock xxmaj west is a of the films films that you you interested . entire time . you what is happen next . xxmaj the xxmaj cage is xxmaj for a xxunk agent as a has a xxunk town . to find out . xxmaj he xxmaj"
4,"are not allowed to attend soccer xxunk , so the nation 's armed forces have been xxunk to save any women who try to enter from

In [ ]:
learn.save_encoder('enc1')

## Use it to train a classifier

For classification, we need to use two set of transforms: one to numericalize the texts and the other to encode the labels as categories. Note that we have to use the same vocabulary as the one used in fine-tuning the language model.

In [ ]:
lm_vocab = dbunch.vocab

In [ ]:
splits = ColSplitter()(df)
x_tfms = [attrgetter("text"), Tokenizer.from_df("text"), Numericalize(vocab=lm_vocab)]
dsrc = DataSource(df, splits=splits, tfms=[x_tfms, [attrgetter("label"), Categorize()]], dl_type=SortedDL)

We once again use a subclass of `TfmdDL` for the dataloaders, since we want to sort the texts (sortish for the training set) by order of lengths. We also use `pad_collate` to create batches form texts of different lengths.

In [ ]:
dbunch = dsrc.databunch(before_batch=pad_input_chunk)

And there is a factory method, once again:

In [ ]:
dbunch = TextDataBunch.from_df(df, text_col="text", text_vocab=lm_vocab, label_col='label', valid_col='is_valid', bs=32)

In [ ]:
dbunch.show_batch(max_n=2, trunc_at=60)

,text,category
0,"xxbos xxmaj raising xxmaj victor xxmaj vargas : a xxmaj review \n\n xxmaj you know , xxmaj raising xxmaj victor xxmaj vargas is like sticking your hands into a big , xxunk bowl of xxunk . xxmaj it 's warm and gooey , but you 're not sure if it feels right . xxmaj try as i might , no",negative
1,"xxbos xxup the xxup shop xxup around xxup the xxup corner is one of the xxunk and most feel - good romantic comedies ever made . xxmaj there 's just no getting around that , and it 's hard to actually put one 's feeling for this film into words . xxmaj it 's not one of those films that",positive


Then we once again have a convenience function to create a classifier from this `DataBunch` with the `AWD_LSTM` architecture.

In [ ]:
learn = text_classifier_learner(dbunch, AWD_LSTM, metrics=[accuracy], path=path,drop_mult=0.5)

In [ ]:
learn = learn.load_encoder('enc1')

Then we can train with gradual unfreezing and differential learning rates.

In [ ]:
learn.fit_one_cycle(4)

epoch,train_loss,valid_loss,accuracy,time
0,0.727597,0.584302,0.745000,00:05
1,0.606741,0.504984,0.745000,00:05
2,0.530358,0.477460,0.765000,00:05
3,0.476057,0.470573,0.745000,00:05


In [ ]:
learn.unfreeze()
learn.opt = learn.create_opt()
learn.fit_one_cycle(8, slice(1e-5,1e-3))

In [ ]:
learn.show_results(max_n=2, trunc_at=60)

In [ ]:
learn.predict("This was a good movie")

In [ ]:
from fastai2.interpret import *

In [ ]:
interp = Interpretation.from_learner(learn)

In [ ]:
interp.plot_top_losses(6)